<a href="https://colab.research.google.com/github/debojit11070/ai-engineering-aws/blob/main/embaddings_for_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from importlib.metadata import version
import tiktoken
import torch

In [2]:
print(version("torch"))

2.6.0+cu124


In [3]:
print(version("tiktoken"))

0.9.0


##Loading the dataset

In [4]:
# Reading the whole file
with open("/content/the-verdict.txt", "r", encoding="latin-1") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
# tokenizing the raw text
import re
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [6]:
len(preprocessed)

4649

##convert tokens into token ids


In [7]:
# finding the unique words
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1159


In [8]:
vocab = {token:integer for integer , token in enumerate(all_words)}

In [9]:
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Carlo;': 25,
 'Chicago': 26,
 'Claude': 27,
 'Come': 28,
 'Croft': 29,
 'Destroyed': 30,
 'Devonshire': 31,
 'Don': 32,
 'Dubarry': 33,
 'Emperors': 34,
 'Florence': 35,
 'For': 36,
 'Gallery': 37,
 'Gideon': 38,
 'Gisburn': 39,
 'Gisburns': 40,
 'Grafton': 41,
 'Greek': 42,
 'Grindle': 43,
 'Grindle:': 44,
 'Grindles': 45,
 'HAD': 46,
 'Had': 47,
 'Hang': 48,
 'Has': 49,
 'He': 50,
 'Her': 51,
 'Hermia': 52,
 'His': 53,
 'How': 54,
 'I': 55,
 'If': 56,
 'In': 57,
 'It': 58,
 'Jack': 59,
 'Jove': 60,
 'Just': 61,
 'Lord': 62,
 'Made': 63,
 'Miss': 64,
 'Money': 65,
 'Monte': 66,
 'Moon-dancers': 67,
 'Mr': 68,
 'Mrs': 69,
 'My': 70,
 'Never': 71,
 'No': 72,
 'Now': 73,
 'Nutley': 74,
 'Of': 75,
 'Oh': 76,
 'On': 

In [10]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i == 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)


In [11]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
                         item.strip() for item in preprocessed if item.strip()
                       ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)

text = """It's the last he painted, you know,
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 69, 7, 39, 873, 1136, 773, 812, 7]


In [13]:
tokenizer.decode(ids)


"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

In [14]:
print(tokenizer.decode(ids))


It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


In [17]:
len(vocab)

1159

In [18]:

# Doing the same preprocessing and creating the vocab as before
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_tokens = sorted(list(set(preprocessed)))

# Adding these two extra tokens
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Carlo;': 25,
 'Chicago': 26,
 'Claude': 27,
 'Come': 28,
 'Croft': 29,
 'Destroyed': 30,
 'Devonshire': 31,
 'Don': 32,
 'Dubarry': 33,
 'Emperors': 34,
 'Florence': 35,
 'For': 36,
 'Gallery': 37,
 'Gideon': 38,
 'Gisburn': 39,
 'Gisburns': 40,
 'Grafton': 41,
 'Greek': 42,
 'Grindle': 43,
 'Grindle:': 44,
 'Grindles': 45,
 'HAD': 46,
 'Had': 47,
 'Hang': 48,
 'Has': 49,
 'He': 50,
 'Her': 51,
 'Hermia': 52,
 'His': 53,
 'How': 54,
 'I': 55,
 'If': 56,
 'In': 57,
 'It': 58,
 'Jack': 59,
 'Jove': 60,
 'Just': 61,
 'Lord': 62,
 'Made': 63,
 'Miss': 64,
 'Money': 65,
 'Monte': 66,
 'Moon-dancers': 67,
 'Mr': 68,
 'Mrs': 69,
 'My': 70,
 'Never': 71,
 'No': 72,
 'Now': 73,
 'Nutley': 74,
 'Of': 75,
 'Oh': 76,
 'On': 

In [20]:
len(vocab)

1161

In [21]:
len(all_tokens)

1161

In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)


In [24]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
                            item if item in self.str_to_int
                            else "<|unk|>" for item in preprocessed
                        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [27]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [28]:
tokenizer.encode(text)

[1160,
 5,
 362,
 1155,
 642,
 1000,
 10,
 1159,
 57,
 1013,
 981,
 1009,
 738,
 1013,
 1160,
 7]

In [30]:
tokenizer.decode(tokenizer.encode(text))


'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

##Bytepair encoding

In [31]:

import tiktoken
print("tiktoken version:", tiktoken.__version__)

tiktoken version: 0.9.0


In [32]:
tokenizer = tiktoken.get_encoding("gpt2")


In [33]:
# vocabular size of gpt-2
tokenizer.n_vocab


50257

In [34]:
tiktoken.list_encoding_names()

['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base', 'o200k_base']

In [35]:
tiktoken.get_encoding("cl100k_base").n_vocab


100277

In [36]:
text = (
            "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
             "of someunknonword."
        )

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 2954, 13159, 4775, 13]


In [37]:
len(integers)


21

In [38]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknonword.


In [39]:
tokenizer.encode('Akwirw ier')

[33901, 86, 343, 86, 220, 959]

In [40]:
tokenizer.decode([33901, 86, 343, 86, 220, 959])


'Akwirw ier'

## data sampling with sliding window

In [41]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)

# The total number of tokens in the training set, after applying the BPE tokenizer.
print(len(enc_text))

5145


In [42]:
enc_sample = enc_text[50:]


In [43]:
enc_sample

[290,
 4920,
 2241,
 287,
 257,
 4489,
 64,
 319,
 262,
 34686,
 41976,
 13,
 357,
 10915,
 314,
 2138,
 1807,
 340,
 561,
 423,
 587,
 10598,
 393,
 28537,
 2014,
 198,
 198,
 1,
 464,
 6001,
 286,
 465,
 13476,
 1,
 438,
 5562,
 373,
 644,
 262,
 1466,
 1444,
 340,
 13,
 314,
 460,
 3285,
 9074,
 13,
 46606,
 536,
 5469,
 438,
 14363,
 938,
 4842,
 1650,
 353,
 438,
 2934,
 489,
 3255,
 465,
 48422,
 540,
 450,
 67,
 3299,
 13,
 366,
 5189,
 1781,
 340,
 338,
 1016,
 284,
 3758,
 262,
 1988,
 286,
 616,
 4286,
 705,
 1014,
 510,
 26,
 475,
 314,
 836,
 470,
 892,
 286,
 326,
 11,
 1770,
 13,
 8759,
 2763,
 438,
 1169,
 2994,
 284,
 943,
 17034,
 318,
 477,
 314,
 892,
 286,
 526,
 383,
 1573,
 11,
 319,
 9074,
 13,
 536,
 5469,
 338,
 11914,
 11,
 33096,
 663,
 4808,
 3808,
 62,
 355,
 996,
 484,
 547,
 12548,
 287,
 281,
 13079,
 410,
 12523,
 286,
 22353,
 13,
 843,
 340,
 373,
 407,
 691,
 262,
 9074,
 13,
 536,
 48819,
 508,
 25722,
 276,
 13,
 11161,
 407,
 262,
 40123,
 18113,


In [44]:

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [45]:

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [46]:

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [47]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


In [48]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [49]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = MyDataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [50]:

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [51]:
dataloader = create_dataloader_v1(
                                    raw_text,
                                    batch_size=1,
                                    max_length=4,
                                    stride=1,
                                    shuffle=False
                                )

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [52]:

second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [53]:
dataloader = create_dataloader_v1(raw_text,
                                  batch_size=8,
                                  max_length=4,
                                  stride=4,
                                  shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## creating token embedding